## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

# ReAct Pattern

In [26]:
import openai
import os
from dotenv import load_dotenv
import io
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [50]:

from openai import OpenAI

openai = OpenAI()

# Request prompt
request = (
    "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
    "Answer only with the question, no explanation."
)



def generate_question(prompt: str) -> str:
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': prompt}]
    )
    question = response.choices[0].message.content
    return question

def react_agent_decide_model(question: str) -> str:
    prompt = f"""
            You are an intelligent AI assistant tasked with evaluating which language model is most suitable to answer a given question.

            Available models:
            - OpenAI: excels at reasoning and factual answers.
            - Claude: better for philosophical, nuanced, and ethical topics.
            - Gemini: good for concise and structured summaries.
            - Groq: good for creative or exploratory tasks.
            - DeepSeek: strong at coding, technical reasoning, and multilingual responses.

            Here is the question to answer:
            "{question}"

            ### Thought:
            Which model is best suited to answer this question, and why?

            ### Action:
            Respond with only the model name you choose (e.g., "Claude").
                """

    response = openai.chat.completions.create(
        model="o3-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    model = response.choices[0].message.content.strip()
    return model

def generate_answer_openai(prompt):
    answer = openai.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{'role': 'user', 'content': prompt}]
        ).choices[0].message.content
    return answer

def generate_answer_anthropic(prompt):
    anthropic = Anthropic(api_key=anthropic_api_key)
    model_name = "claude-3-5-sonnet-20240620"
    answer = anthropic.messages.create(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}],
            max_tokens=1000
        ).content[0].text
    return answer

def generate_answer_deepseek(prompt):
    deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
    model_name = "deepseek-chat"    
    answer = deepseek.chat.completions.create(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}],
            base_url='https://api.deepseek.com/v1'
        ).choices[0].message.content
    return answer

def generate_answer_gemini(prompt):
    gemini=OpenAI(base_url='https://generativelanguage.googleapis.com/v1beta/openai/',api_key=google_api_key)
    model_name = "gemini-2.0-flash"
    answer = gemini.chat.completions.create(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}],
        ).choices[0].message.content
    return answer

def generate_answer_groq(prompt):
    groq=OpenAI(base_url='https://api.groq.com/openai/v1',api_key=groq_api_key)
    model_name="llama3-70b-8192"
    answer = groq.chat.completions.create(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}],
            base_url="https://api.groq.com/openai/v1"
        ).choices[0].message.content
    return answer

def main():
    print("Generating question...")
    question = generate_question(request)
    print(f"\n🧠 Question: {question}\n")
    selected_model = react_agent_decide_model(question)
    print(f"\n🔹 {selected_model}:\n")
    
    if selected_model.lower() == "openai":
        answer = generate_answer_openai(question)
    elif selected_model.lower() == "deepseek":
        answer = generate_answer_deepseek(question)
    elif selected_model.lower() == "gemini":
        answer = generate_answer_gemini(question)
    elif selected_model.lower() == "groq":
        answer = generate_answer_groq(question)
    elif selected_model.lower() == "claude":
        answer = generate_answer_anthropic(question)
    print(f"\n🔹 {selected_model}:\n{answer}\n")
    


In [ ]:
main()

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>